#ARS TLS practical part 3: Quantitative structural model (QSM) generation with raycloudtools


By now, we have an individual tree that has been automatically segmented and manually corrected in CloudCompare. Now that we are sure the point cloud of the tree is of sufficient quality, we can reapply RayCloudTools (RCT) Quantitative structure model (QSM) generation to acquire a more accurate structural model of the tree. A Quantitative Structure Model (QSM) is a 3D geometric representation of a tree that approximates its woody structure using simple shapes (usually cylinders) fitted to point cloud data.

This part of the lesson is very short, but still pay attention to how the same tools from RCT that we previously used for terrain extraction and segmentation are now used to model a singular tree.

Before getting started, remount the google drive and install rct/treetools for this session. This is a great time to take a break during the ~10 minute loading process.

In [ ]:
# ==========================================================
# Mount Google Drive (run once per session)
# ==========================================================

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/
!sudo apt-get install libeigen3-dev
!git clone https://github.com/ethz-asl/libnabo.git
%cd libnabo
!ls
!git checkout tags/1.0.7
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE=RelWithDebInfo
!make
!sudo make install
%cd ../..
!git clone http://github.com/qhull/qhull.git
%cd qhull
!git checkout tags/v7.3.2
!mkdir build
%cd build
!cmake .. -DCMAKE_POSITION_INDEPENDENT_CODE:BOOL=true
!make
!sudo make install
%cd ../..
!ls
!git clone https://github.com/csiro-robotics/raycloudtools.git
%cd raycloudtools
!mkdir build
%cd build
!cmake .. -DWITH_QHULL=ON
!sudo make install
%cd ../..
#extra step to modify cmake target file for google colab file structure, allows treetools to find raycloudtools install
with open("/content/raycloudtools/build/raycloudtools-config.cmake","r",encoding='utf-8') as file:
  data = file.readlines()
data[27] = 'include("/content/raycloudtools/build/raylib/raycloudtools-targets.cmake")\n'
with open("/content/raycloudtools/build/raycloudtools-config.cmake","w",encoding='utf-8') as file:
  file.writelines(data)
!git clone https://github.com/csiro-robotics/treetools.git
%cd treetools
!mkdir build
%cd build
!cmake .. -DCMAKE_PREFIX_PATH=/content/raycloudtools/build/
!sudo make install
%cd ../..

# Step 1: Import our individual tree point cloud with rayimport

The following block is designed to work with a file structure in the google drive that contains a single tree .ply point cloud within a folder 'corrected tree'. You can either follow that structure or modify the code to follow your structure.

In [ ]:
#import the individual tree
#call rayimport
%cd /content/
!/content/raycloudtools/build/bin/rayimport /content/drive/MyDrive/ARS_TLS_course/data/corrected_tree/corrected_tree_29.ply ray 0,0,-0.1 --max_intensity 0

#Step 2: Apply rayextract terrain/trees to the raycloud.ply

These steps probably seem familiar to the automated segmentation we previously applied. However, if you examine the created mesh file, you will notice that it is only a small mesh created directly under the tree. Rayextract trees requires a terrain mesh input, so we create this small mesh to ensure the tree.txt file - that is the file containing our structural data - can be created.

In [ ]:
#run the individual tree rayextract terrain/trees method
!/content/raycloudtools/build/bin/rayextract terrain /content/drive/MyDrive/ARS_TLS_course/data/corrected_tree/corrected_tree_29_raycloud.ply
!/content/raycloudtools/build/bin/rayextract trees /content/drive/MyDrive/ARS_TLS_course/data/corrected_tree/corrected_tree_29_raycloud.ply /content/drive/MyDrive/ARS_TLS_course/data/corrected_tree/corrected_tree_29_raycloud_mesh.ply --max_diameter 0.9 --height_min 10

> 🤔 **Reflection 1**: Why do you think it is necessary to create a mesh to continue the process? That is - what aspect of the raycloudtools structure modeling approach do you think is dependant on the terrain mesh to model a tree?

#Step 3: Investigate the results and troubleshoot errors

Now we should have a mesh, pointcloud, and tree.txt file generated from this process. While these files are generated in the same way as the plot level automated segmentation, the input data limited the scope of the model run to focus it on an individual tree. If everything ran correctly, we should have a small mesh file, a .ply point cloud with a single class for one 'segmented' tree, and a tree.txt file containing one line with all the structural information for our tree.

##Troubleshooting
The most common error with this individual tree modeling method is that it will generate multiple trees from the input point cloud, even though it looks like you clearly input a single tree. To troubleshoot this, you can either:
1. Modify the input parameters of rayextract terrain/trees to guide the model towards modeling a single tree, or
2. Modify your input data to make it easier for rct to model a single tree.

This process will take trial and error, and you will need to build a sense of how the model determines its behavior. Keep in mind, the model is not stochastic, meaning that it will output the same results given the same input parameters and data.

> 🤔 **Reflection 2**: You have now completed all the the tree modeling and point cloud manipulation in the lesson and will move on to extracting metrics from the data. Keep in mind that in actual practice, we would want to apply this process to potentially hundreds or thousands of trees. How would you change or alter the workflow presented here to make it applicable on a much wider scale?